## Add utils as a module and import

In [3]:
import sys
    
try:
    from utils import data_loading
except:
    sys.path.append('../utils')
    sys.path.append('../')
    from utils import data_loading

# Loading libraries

In [4]:
import os
import numpy as np
from utils import model
from pathlib import Path
from PIL import Image'
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr
from sklearn.decomposition import PCA

# Loading Data

In [5]:
#loading data for subject 1, first 10 images
lh, rh, images  = data_loading.load_subject_data(1, 0, 200)

print(lh.shape)
print(rh.shape)
print(images[0].shape)

(200, 19004)
(200, 20544)
(425, 425, 3)


In [6]:
#concat lh and rh
brain = np.concatenate((lh, rh), axis=1)
brain_separate = [lh, rh]
print(len(brain_separate))

2


In [9]:
dataset = data_loading.CustomDataset(images_list= images, outputs_list= brain, transform=transforms.ToTensor(), PCA = PCA(n_components = 100))

In [9]:
#Create a train and validation subset of variable dataset with torch
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
#put train dataset into a loader with 2 batches and put test data in val loader
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(test_dataset, shuffle=True)

In [10]:
reg_model = model.ResNet2HeadModel(100)
trainer = model.Trainer()
optimizer = torch.optim.Adam
loss = torch.nn.MSELoss()
trainer.compile(reg_model, optimizer, learning_rate=0.0001, loss_fn=loss)

In [12]:
trainer.fit(num_epochs= 2, train_loader=train_loader, val_loader=val_loader)

ValueError: too many values to unpack (expected 2)

In [2]:
trainer.history

NameError: name 'trainer' is not defined

In [14]:
trainer.save('initial_model.pt')

Model and training history saved to initial_model.pt


In [16]:
del trainer

In [11]:
trainer  = model.Trainer()
trainer.compile(reg_model, optimizer, learning_rate=0.0001, loss_fn=loss)

In [28]:
trainer.load('initial_model.pt')

Model and training history loaded from initial_model.pt


In [29]:
trainer.history

{'train_loss': [170.70301513671876, 167.72671508789062],
 'val_loss': [183.5020439147949, 183.56376399993897]}

In [7]:
from sklearn.decomposition import PCA
import torch
from torch.utils.data import Dataset
from PIL import Image

class CustomDataset1(Dataset):
    def __init__(self, images_list, outputs_list1, outputs_list2, transform=None, PCA1=None, PCA2=None):
        self.num_samples = len(images_list)
        self.transform = transform
        self.PCA1 = PCA1
        self.PCA2 = PCA2
        self.data, self.output1, self.output2 = self.load_data(images_list, outputs_list1, outputs_list2)

    def load_data(self, images_list, outputs_list1, outputs_list2):
        data = []
        output_concat1 = []
        output_concat2 = []

        for i in range(self.num_samples):
            # Load image from the given list
            image = Image.fromarray(images_list[i])

            # Load output arrays from the given lists
            output1 = outputs_list1[i]
            output2 = outputs_list2[i]

            data.append((image, output1, output2))
            output_concat1.append(output1)
            output_concat2.append(output2)

        if self.PCA1:
            self.PCA1.fit(output_concat1)

        if self.PCA2:
            self.PCA2.fit(output_concat2)

        return data, output_concat1, output_concat2

    def give_output(self):
        return self.output1, self.output2

    def get_PCA1(self):
        return self.PCA1

    def get_PCA2(self):
        return self.PCA2

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        image, output1, output2 = self.data[idx]

        if self.transform:
            image = self.transform(image)

        if self.PCA1:
            output1 = self.PCA1.transform(output1.reshape(1, -1))

        if self.PCA2:
            output2 = self.PCA2.transform(output2.reshape(1, -1))

        return image, torch.FloatTensor(output1[0]), torch.FloatTensor(output2[0])

In [8]:
dataset = CustomDataset1(images_list= images, outputs_list1= brain_separate[0], outputs_list2 = brain_separate[1], transform=transforms.ToTensor(), PCA1 = PCA(n_components = 100), PCA2 = PCA(n_components = 100))